In [1]:
import requests
import cv2
import glob
import os, sys
import json
url = "https://kapi.kakao.com/v1/vision/face/detect"
MYAPI_KEY = '5a826bf53f8508d2cbf62d2199a967dd'

In [2]:
def face_detector(filename):
    headers = {'Authorization': 'KakaoAK {}'.format(MYAPI_KEY)}
    try:
        files = {'file' : open(filename, 'rb')}
        resp = requests.post(url, headers=headers, files=files)
        resp.raise_for_status()
        return resp.json()
    except Exception as e:
        print(str(e))
        sys.exit(0)

In [3]:
def get_key_image_info() :
    key_info = {}
    # ========== read json file from key image info ========== 
    with open('imageKey_result.json', 'r') as json_file:
        key_data = json.load(json_file)
        #print(key_data['result'])
    # ========== save key image info ==========
    key_face =  key_data["result"]["faces"][0]
    key_facial_points = key_face['facial_points'] 
    for each_obj in target_obj:
        for each in key_facial_points[each_obj]:
            key_info[each_obj] = each
            #print("each", each)
    #print("key info dict", key_info)
    return key_info

In [7]:
name = input("What's your name? ")
dirName = "./test/" + name
print('Directory ' + dirName)
if not os.path.exists(dirName):
    os.makedirs(dirName)
    print("Directory Created")
else:
    print("Name already exists")
    sys.exit()

webcam = cv2.VideoCapture(0) 
webcam_info = {}
while True : 
    ret, frame = webcam.read()
    if ret :
        fileName = dirName + "/" + name + ".png"
        cv2.imwrite(fileName,frame) # 이미지저장 
        height, width, d = frame.shape
        files = glob.glob('*.png')
        for file in files:
            json_data = face_detector(file)
            # ========== write all facial info in json file ==========
            with open('webcam_result.json', 'w') as json_file:
                json.dump(json_data, json_file)
            face =  json_data["result"]["faces"][0]
            x = int(face['x'] * width)
            y = int(face['y'] * height)
            w = int(face['w'] * width)
            h = int(face['h'] * height)            
            # ========== facial points like right eye, left eye, etc. ==========
            facial_points = face['facial_points'] 
            # ========== draw rectange & resize frame ========== 
            cv2.rectangle(frame,(x,y),((x+w),(y+h)),(255, 0, 0),3)
            resize_frame = cv2.resize(frame, dsize=(640, 480), interpolation=cv2.INTER_AREA)
            # ========== save webcam img info in dict ==========
            target_obj = ['right_eyebrow', 'left_eyebrow', 'jaw', 'right_eye', 'left_eye', 'nose', 'lip']
            fig_w, fig_h = json_data['result']['width'], json_data['result']['height']
            # ========== get key img info ========== 
            key_img_info = get_key_image_info()
            #print("key info dict", key_img_info)
            # ========== store webcam img info ========== 
            target_obj = ['right_eyebrow', 'left_eyebrow', 'jaw', 'right_eye', 'left_eye', 'nose', 'lip']
            for each_obj in target_obj:
                for each in facial_points[each_obj]:
                    webcam_info[each_obj] = each
            #print("webcam info dict", webcam_info)
            with open('webcamImg_result.json', 'w') as json_file:
                json.dump(webcam_info, json_file)
            # ========== compare values of key image & webcam image ==========           
            for k, v in key_img_info.items() :
                for wk, wv in webcam_info.items() :
                    first_diff = (abs(v[0]-wv[0]) / v[0]) * 100
                    second_diff = (abs(v[1]-wv[1]) / v[1]) * 100
            print('first', first_diff)
            print('second', second_diff)
            avg = 100 - ((first_diff + second_diff) / 2)
            print('avg', avg)
            if (avg >= 80) :
                cv2.circle(img=resize_frame, center=(int(each[0]*fig_w), int(each[1]*fig_h)), radius=3, color=(0, 255, 0), thickness=-1)
                answer = "{} {:.2f}%".format(name, avg)
                cv2.putText(resize_frame, answer, (x-10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            else :
                cv2.putText(resize_frame, 'not recognized', (x-10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        # ========== Display the resulting frame ========== 
        cv2.imshow('KAKAO VISION API EXAMPLE', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
webcam.release()
cv2.destroyAllWindows()

What's your name? hy
Directory ./test/hy
Directory Created
first 2.917324541611557
second 17.92781113022068
avg 89.57743216408389
first 2.9173215305092195
second 17.92781445985219
avg 89.5774320048193
first 2.917324541611557
second 17.92781113022068
avg 89.57743216408389
first 2.9173215305092195
second 17.92781445985219
avg 89.5774320048193
first 2.9173215305092195
second 17.92781445985219
avg 89.5774320048193
first 2.9173215305092195
second 17.92781445985219
avg 89.5774320048193
first 2.917324541611557
second 17.92781113022068
avg 89.57743216408389
first 2.9173215305092195
second 17.92781445985219
avg 89.5774320048193
first 2.917324541611557
second 17.92781113022068
avg 89.57743216408389
first 2.917324541611557
second 17.92781113022068
avg 89.57743216408389
first 2.9173215305092195
second 17.92781445985219
avg 89.5774320048193
first 2.9173215305092195
second 17.92781445985219
avg 89.5774320048193
first 2.917324541611557
second 17.92781113022068
avg 89.57743216408389
first 2.9173245416